In [18]:
import numpy as np
import pandas as pd
import os
from human_eval.data import write_jsonl, read_problems, extract_python

In [19]:
df = pd.read_json("data/samples_codellama.jsonl", lines=True)
df["full_response"]

0      Here is the complete code:\n```python\nimport ...
1      Plan:\n\n1. Import the necessary libraries for...
2      [PYTHON]\nimport numpy as np\nfrom skimage imp...
3      Here is a possible implementation of the bland...
4      Here's the code for the function `combine_colu...
                             ...                        
565    Here is the complete code for the function `wo...
566    Plan:\n\n1. Convert the input image to graysca...
567    Here is the complete code for the function `wo...
568    ```python\nimport numpy as np\nimport pandas a...
569    Here is the complete code for the function `wo...
Name: full_response, Length: 570, dtype: object

In [20]:
directory = "data/"

collection = []
for filename in os.listdir(directory):
    if filename.endswith("_results.jsonl"):
#        print(filename)
        df = pd.read_json(directory + filename, lines=True)
        df['model'] = filename.replace(".jsonl_results.jsonl", "").replace("samples_", "")
        collection.append(df)

df = pd.concat(collection)


In [21]:
test = """
hallo
```
code
```
bla
```
code2
```
"""
extract_python(test)

'\ncode\n\ncode2\n'

In [23]:
# Function-mapping to extract python code from full_response
tmp_dir = "tmp_data/"
if not os.path.exists(tmp_dir):
    os.mkdir(tmp_dir)

df["completion"] = df["full_response"].astype(str).apply(extract_python)
grouped = df.groupby("model")

for name, group in grouped:
    records = group.drop("model", axis=1).to_dict("records")
    write_jsonl(f"{tmp_dir}/samples_{name}.jsonl_results.jsonl", records)

# Testing code after 
for filename in os.listdir("data"):
    if filename.endswith("results.jsonl"):
        model_id = filename.split(".jsonl")[0]
        df_orig = pd.read_json(directory+filename, lines=True)
        df_new = pd.read_json(f"{tmp_dir}{model_id}.jsonl_results.jsonl", lines=True)
        if not df_orig.equals(df_new):
            # checks inequalities
            num_diff = np.size(np.where(~np.all((df_orig == df_new).to_numpy(), axis=1)))
            print(f"{model_id} is different in {num_diff} entries")

samples_llama3-70b-instruct-q4_0 is different in 64 entries
samples_command-r-plus-104b-q4_0 is different in 1 entries
samples_llama3-8b-instruct-fp16 is different in 248 entries
samples_llama3-70b-instruct-q8_0 is different in 94 entries
